<h1><center><b>Лабораторна робота №2</b></center></h1>
<h6>ФБ-34 Кувавіна Софія</h6>

<h5><b><center>Хід роботи:</center></b></h5>

In [17]:
pip install numpy pandas matplotlib ipython

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Sofiya\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import os
import datetime
import pandas as pd
import urllib.request

print("Setup Complete")

Setup Complete


### Завдання 1
Для кожної із адміністративних одиниць України завантажити тестові
структуровані файли, що містять значення VHI-індексу. Ця процедура має бути
автоматизована, параметром процедури має бути індекс (номер) області. При
зберіганні файлу до його імені потрібно додати дату та час завантаження.

In [4]:

def download_data(province_ID, start_year=2000, end_year=2025): 
    os.makedirs("lab2_VHI", exist_ok=True)
    
    # перевіряю чи файл вже існує
    existing_files = [f for f in os.listdir("lab2_VHI") if f.startswith(f"VHI-ID_{province_ID}_")]
    if existing_files:
        print(f" Файл для VHI-ID №{province_ID} вже існує: {existing_files[0]}")
        return os.path.join("lab2_VHI", existing_files[0])

    # завантажую дані
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={province_ID}&year1={start_year}&year2={end_year}&type=Mean"
    filename = f"VHI-ID_{province_ID}_{datetime.datetime.now().strftime('%d-%m-%Y_%H-%M-%S')}.csv"
    
    with open(os.path.join("lab2_VHI", filename), 'wb') as file:
        file.write(urllib.request.urlopen(url).read())

    print(f" VHI-файл {filename} завантажений успішно!")
    return os.path.join("lab2_VHI", filename)

# Завантажую дані для всіх областей + Київ та Севастополь
for i in range(1, 28):
    download_data(i)

 Файл для VHI-ID №1 вже існує: VHI-ID_1_11-03-2025_23-06-48.csv
 Файл для VHI-ID №2 вже існує: VHI-ID_2_11-03-2025_23-06-51.csv
 Файл для VHI-ID №3 вже існує: VHI-ID_3_11-03-2025_23-06-52.csv
 Файл для VHI-ID №4 вже існує: VHI-ID_4_11-03-2025_23-06-53.csv
 Файл для VHI-ID №5 вже існує: VHI-ID_5_11-03-2025_23-06-54.csv
 Файл для VHI-ID №6 вже існує: VHI-ID_6_11-03-2025_23-06-55.csv
 Файл для VHI-ID №7 вже існує: VHI-ID_7_11-03-2025_23-06-55.csv
 Файл для VHI-ID №8 вже існує: VHI-ID_8_11-03-2025_23-06-56.csv
 Файл для VHI-ID №9 вже існує: VHI-ID_9_11-03-2025_23-06-57.csv
 Файл для VHI-ID №10 вже існує: VHI-ID_10_11-03-2025_23-06-58.csv
 Файл для VHI-ID №11 вже існує: VHI-ID_11_11-03-2025_23-06-58.csv
 Файл для VHI-ID №12 вже існує: VHI-ID_12_11-03-2025_23-06-59.csv
 Файл для VHI-ID №13 вже існує: VHI-ID_13_11-03-2025_23-07-00.csv
 Файл для VHI-ID №14 вже існує: VHI-ID_14_11-03-2025_23-07-01.csv
 Файл для VHI-ID №15 вже існує: VHI-ID_15_11-03-2025_23-07-01.csv
 Файл для VHI-ID №16 вже існ

1) Зчитати завантажені текстові файли у фрейм. 
2) Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). 
3) Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [18]:
def dataframer(folder_path):
    fr, columns  = [], ["Year", "Week", "SMN", "SMT", "VCI", "TCI", "VHI", "empty"]

    # отримую список файлів у папці
    csv_files = filter(lambda x: x.endswith('.csv'), os.listdir(folder_path))

    # далі перебираю файли
    for file_name in csv_files:
        file_path = os.path.join(folder_path, file_name)
        province_ID = int(file_name.split('_')[1])
        
        # зчитую та обробляю файли
        df = pd.read_csv(file_path, header=1, names=columns) 
        df.at[0, "Year"] = df.at[0, "Year"][9:] 

        df = df.drop(df.index[-1]) 
        df = df.drop(df.loc[df["VHI"] == -1].index) # видаляю рядки з невизначеним VHI
        df = df.drop("empty", axis=1) 

        df.insert(0, "province_ID", province_ID, True) 
        df['Year'] = df['Year'].astype(int)
        df["Week"] = df["Week"].astype(int) 

        fr.append(df) # додаю до списку

    # об'єдную дані у фрейм видаливши дублікати
    df_res = pd.concat(fr).drop_duplicates().reset_index(drop=True)
    

    return df_res


def format_result(df):
    result_string = ""
    for province_ID, province_data in df.groupby('province_ID'):
        # для кожної області обираю приклад даних початкового та останнього року
        start_year_example_data = province_data[province_data['Year'] == province_data['Year'].min()].head(1)
        end_year_example_data = province_data[province_data['Year'] == province_data['Year'].max()].tail(1)
        
        result_string += "\n\n\n" + " "*17 + f"[= Province ID: {province_ID} =]\n"
        result_string += "=" * 55 + "\n"
        
        
        result_string += " "*12 + "Приклад даних початкового року:\n"
        result_string += "-" * 55 + "\n"
        result_string += f"province_ID | Year | Week | SMN | SMT | VCI | TCI | VHI\n"
        result_string += "-" * 55 + "\n"
        result_string += start_year_example_data.to_string(index=False, header=False).replace('\n', f'\n{province_ID}| ').replace(' ', ' | ') + "\n"
        result_string += " " * 26 + "..." + " " * 26 + "\n"
        
        result_string += " "*12 + "Приклад даних останнього року:\n"
        result_string += "-" * 55 + "\n"
        result_string += f"province_ID | Year | Week | SMN | SMT | VCI | TCI | VHI\n"
        result_string += "-" * 55 + "\n"
        result_string += end_year_example_data.to_string(index=False, header=False).replace('\n', f'\n{province_ID}| ').replace(' ', ' | ') + "\n"
        
    return result_string

# приклад
folder_path = "lab2_VHI"
data_frame = dataframer(folder_path)
result_string = format_result(data_frame)
print(result_string)




                 [= Province ID: 1 =]
            Приклад даних початкового року:
-------------------------------------------------------
province_ID | Year | Week | SMN | SMT | VCI | TCI | VHI
-------------------------------------------------------
1 | 2000 | 1 | 0.031 | 257.0 | 20.04 | 51.54 | 35.79
                          ...                          
            Приклад даних останнього року:
-------------------------------------------------------
province_ID | Year | Week | SMN | SMT | VCI | TCI | VHI
-------------------------------------------------------
1 | 2025 | 9 | 0.111 | 274.35 | 61.72 | 23.46 | 42.57



                 [= Province ID: 2 =]
            Приклад даних початкового року:
-------------------------------------------------------
province_ID | Year | Week | SMN | SMT | VCI | TCI | VHI
-------------------------------------------------------
2 | 2000 | 1 | 0.044 | 262.34 | 18.62 | 31.06 | 24.84
                          ...                          
          

Реалізувати окрему процедуру, яка змінить індекси областей, які використані на
порталі NOAA (за англійською абеткою) на наступні, за українською (виключно
старі індекси на нові):

In [7]:
def change_province_id(df):
    # словник між індексами областей
    province_mapping = {
        1: 22,
        2: 24,
        3: 23,
        4: 25,
        5: 3,
        6: 4,
        7: 8,
        8: 19,
        9: 20,
        10: 21,
        11: 9,
        13: 10,
        14: 11,
        15: 12,
        16: 13,
        17: 14,
        18: 15,
        19: 16,
        21: 17,
        22: 18,
        23: 6,
        24: 1,
        25: 2,
        26: 7,
        27: 5,
    }
    
    # міняю індекси областей у фреймі
    df_copy = df.copy() 
    df_copy['province_ID'] = df_copy['province_ID'].replace(province_mapping)
    
    return format_result(df_copy)

# приклад
folder_path = "lab2_VHI"
data_frame = dataframer(folder_path)
changed_id_result = change_province_id(data_frame)
print(changed_id_result)




                 [= Province ID: 1 =]
            Приклад даних початкового року:
-------------------------------------------------------
province_ID | Year | Week | SMN | SMT | VCI | TCI | VHI
-------------------------------------------------------
1 | 2000 | 1 | 0.023 | 260.25 | 8.46 | 39.97 | 24.22
                          ...                          
            Приклад даних останнього року:
-------------------------------------------------------
province_ID | Year | Week | SMN | SMT | VCI | TCI | VHI
-------------------------------------------------------
1 | 2025 | 9 | 0.132 | 278.16 | 75.21 | 14.09 | 44.65



                 [= Province ID: 2 =]
            Приклад даних початкового року:
-------------------------------------------------------
province_ID | Year | Week | SMN | SMT | VCI | TCI | VHI
-------------------------------------------------------
2 | 2000 | 1 | 0.041 | 264.68 | 13.12 | 36.19 | 24.65
                          ...                          
          

<p><b> Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу): </b></p>
<p>1. Ряд VHI для області за рік;</p>

In [8]:
def get_vhi_series(df, province_id, year):
    vhi_series = df[(df['province_ID'] == province_id) & (df['Year'] == year)]
    
    print("\n" + "="*60)
    print(f"🔹 Ряд VHI для області {province_id} за {year} рік")
    print("="*60)
    print(vhi_series.to_string(index=False))
    
    return vhi_series

# Приклад 
folder_path = "lab2_VHI"
data_frame = dataframer(folder_path)  

vhi_series_df = get_vhi_series(data_frame, province_id=10, year=2023)


🔹 Ряд VHI для області 10 за 2023 рік
 province_ID  Year  Week   SMN    SMT   VCI   TCI   VHI
          10  2023     1 0.050 257.52 39.28 45.60 42.44
          10  2023     2 0.054 257.02 44.86 48.34 46.60
          10  2023     3 0.056 256.81 49.13 49.76 49.45
          10  2023     4 0.062 258.12 53.87 46.39 50.13
          10  2023     5 0.071 261.50 57.38 40.51 48.95
          10  2023     6 0.083 265.68 59.46 33.80 46.63
          10  2023     7 0.099 269.90 65.10 25.92 45.51
          10  2023     8 0.117 272.50 71.09 23.64 47.36
          10  2023     9 0.134 274.39 75.48 24.02 49.75
          10  2023    10 0.153 276.91 78.56 22.14 50.35
          10  2023    11 0.167 279.02 77.55 20.02 48.78
          10  2023    12 0.182 280.71 76.01 22.93 49.47
          10  2023    13 0.192 282.28 71.24 26.93 49.08
          10  2023    14 0.207 283.96 67.24 29.76 48.50
          10  2023    15 0.231 285.92 65.35 34.04 49.70
          10  2023    16 0.259 288.12 63.84 38.09 50.96
          

2) Пошук екстремумів (min та max) для вказаних областей та років,
середнього, медіани;

In [9]:
def calculate_vhi_statistics(df, start_year=None, end_year=None):
    # фільтрую за роками, якщо вони задані
    if start_year is not None:
        df = df[df['Year'] >= start_year]
    if end_year is not None:
        df = df[df['Year'] <= end_year]

    # групую дані за областю та роком
    grouped_data = df.groupby(['province_ID', 'Year'])

    vhi_statistics = []

    # проходжусь по групах
    for (province_id, year), group_df in grouped_data:
        min_vhi = group_df['VHI'].min()
        max_vhi = group_df['VHI'].max()
        mean_vhi = group_df['VHI'].mean()
        median_vhi = group_df['VHI'].median()

        vhi_statistics.append({
            'province_ID': province_id, 
            'Year': year, 
            'Min_VHI': min_vhi, 
            'Max_VHI': max_vhi, 
            'Mean_VHI': mean_vhi, 
            'Median_VHI': median_vhi
        })

    result_df = pd.DataFrame(vhi_statistics)

    print("\n" + "="*80)
    print(" Мінімальні, максимальні, середні та медіанні значення VHI")
    print("="*80)
    print(result_df.to_string(index=False))

    return result_df

# приклад 
folder_path = "lab2_VHI"
data_frame = dataframer(folder_path)  

vhi_statistics_df = calculate_vhi_statistics(data_frame, start_year=2005, end_year=2025)


 Мінімальні, максимальні, середні та медіанні значення VHI
 province_ID  Year  Min_VHI  Max_VHI  Mean_VHI  Median_VHI
           1  2005    41.62    68.89 52.226275      50.650
           1  2006    35.84    66.33 49.301154      47.140
           1  2007    26.46    52.82 41.131154      42.410
           1  2008    25.87    74.45 47.917500      46.995
           1  2009    36.76    62.66 45.313846      43.920
           1  2010    28.04    55.12 43.785962      44.415
           1  2011    35.62    72.75 46.720192      42.020
           1  2012    38.14    53.09 43.263077      42.340
           1  2013    30.00    58.45 48.292500      48.060
           1  2014    37.59    67.44 49.257500      46.380
           1  2015    34.29    63.73 45.405192      42.885
           1  2016    37.99    68.99 51.801731      50.675
           1  2017    35.58    59.50 45.551731      46.040
           1  2018    39.49    70.83 50.599423      48.195
           1  2019    18.02    69.20 43.484231      40.

3) Ряд VHI за вказаний діапазон років для вказаних областей;

In [ ]:

def get_vhi_by_year_range(df, province_ids, start_year, end_year):

    # фільтрую дані
    result = df[(df["province_ID"].isin(province_ids)) & (df["Year"].between(start_year, end_year))]

    if result.empty:
        print("\nДані для  діапазону відсутні\n")
        return pd.DataFrame()  

    
    print("\n" + "="*80)
    print(f" Дані VHI для областей {province_ids} у період {start_year}-{end_year}")
    print("="*80)
    print(result[["Year", "Week", "province_ID", "VHI"]].to_string(index=False))

    return result[["Year", "Week", "province_ID", "VHI"]]

# приклад
folder_path = "lab2_VHI"
data_frame = dataframer(folder_path)  
province_ids = [3, 4]  
start_year = 2005  
end_year = 2019 

vhi_range_data = get_vhi_by_year_range(data_frame, province_ids, start_year, end_year)




 Дані VHI для областей [3, 4] у період 2005-2019
 Year  Week  province_ID   VHI
 2005     1            3 58.56
 2005     2            3 58.17
 2005     3            3 57.93
 2005     4            3 56.31
 2005     5            3 55.12
 2005     6            3 55.53
 2005     7            3 55.52
 2005     8            3 56.71
 2005     9            3 58.18
 2005    10            3 58.16
 2005    11            3 56.41
 2005    12            3 54.59
 2005    13            3 52.91
 2005    14            3 51.27
 2005    15            3 48.86
 2005    16            3 47.07
 2005    18            3 53.93
 2005    19            3 56.75
 2005    20            3 53.90
 2005    21            3 59.28
 2005    22            3 59.89
 2005    23            3 61.36
 2005    24            3 61.12
 2005    25            3 60.61
 2005    26            3 60.48
 2005    27            3 59.96
 2005    28            3 60.65
 2005    29            3 63.13
 2005    30            3 61.80
 2005    31         

4) Для всього набору даних виявити роки, протягом яких екстремальні
посухи торкнулися більше вказаного відсотка областей по Україні (20%
областей - 5 областей з 25). Повернути роки, назви областей з
екстремальними посухами та значення VHI;

In [20]:
def drought_years(df, percentage, type_of_drought):
    # словник кількості областей з VHI
    drought_count = {}

    grouped_data = df.groupby('Year')

    # перебираю кожний рік
    for year, group_df in grouped_data:
        total_areas = len(group_df['province_ID'].unique())

        if type_of_drought == "extreme":
            drought_areas = group_df[group_df['VHI'] < 15]['province_ID'].nunique()
        elif type_of_drought == "moderate":
            drought_areas = group_df[(group_df['VHI'] > 15) & (group_df['VHI'] < 35)]['province_ID'].nunique()
        
        if drought_areas / total_areas > percentage / 100:
            drought_count[year] = (drought_areas, round((drought_areas / total_areas) * 100, 2))

    if type_of_drought == "extreme":
        result_table = f"Роки з екстремальною посухою (де більше {percentage}% областей мають VHI < 15):\n"
    elif type_of_drought == "moderate":
        result_table = f"Роки з помірною посухою (де більше {percentage}% областей мають VHI від 15 до 35):\n"

    result_table += "-" * 60 + "\n"
    result_table += f"{'Year':^10} | {'К-сть областей з посухою':^25} | {'Відсоток':^15}\n"
    result_table += "-" * 60 + "\n"

    for year, (drought_areas, percent) in drought_count.items():
        result_table += f"{year:^10} | {drought_areas:^25} | {percent:^15}%\n"

    result_table += "-" * 60 + "\n"

    return result_table

# приклад для екстримальних
drought_years_extreme = drought_years(data_frame, 20, "extreme")
print(drought_years_extreme)

# приклад для помірних
drought_years_moderate = drought_years(data_frame, 20, "moderate")
print(drought_years_moderate)

Роки з екстремальною посухою (де більше 20% областей мають VHI < 15):
------------------------------------------------------------
   Year    | К-сть областей з посухою  |    Відсоток    
------------------------------------------------------------
   2000    |             6             |      22.22     %
------------------------------------------------------------

Роки з помірною посухою (де більше 20% областей мають VHI від 15 до 35):
------------------------------------------------------------
   Year    | К-сть областей з посухою  |    Відсоток    
------------------------------------------------------------
   2000    |            27             |      100.0     %
   2001    |            12             |      44.44     %
   2002    |            12             |      44.44     %
   2003    |            13             |      48.15     %
   2005    |             7             |      25.93     %
   2006    |            13             |      48.15     %
   2007    |            15     